In [50]:
import os, sys, argparse
import pandas as pd
import numpy as np

import pickle
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

import numba
import sklearn
import sklearn.ensemble
import sklearn.metrics
import sklearn.naive_bayes
import sklearn.linear_model
import xgboost

import warnings
# warnings.filterwarnings(action='ignore')

In [65]:
parser = argparse.ArgumentParser(description='non socks rtv')
parser.add_argument('-d', '--data', type=str, default='alpha', choices=['alpha', 'amazon', 'epinions', 'otc'], help='data name', required=True)
parser.add_argument('-t', '--tnum', type=int, default=100, help='trusted size')
parser.add_argument('-v', '--vnum', type=int, default=500, help='verified size')

if bool(getattr(sys, 'ps1', sys.flags.interactive)):
    from tqdm import tqdm_notebook as tqdm
    print('interactive mode')
    parsed = parser.parse_args(['--d', 'otc', '--tnum', '100', '--vnum', '500'])
else:
    from tqdm import tqdm
    print('script mode')
    parsed = parser.parse_args(sys.argv[1:])
    display=print

print(parsed)

dname = parsed.data
tnum = parsed.tnum
vnum = parsed.vnum

interactive mode
Namespace(data='otc', tnum=100, vnum=500)


In [66]:
a1_list = range(1, 4)
a2_list = range(1, 4)

b1_list = range(1, 4)
b2_list = range(1, 4)

g1_list = range(20, 100, 20)
g2_list = range(10, 20, 5)
g3_list = range(1, 10, 5)
g4_list = range(1, 10, 5)

template = '../res/non-socks/rtv-%s-%d-%d-%d-%d-%d-%d-%d-%d.csv'
maxiter = 30

flist = []
for a1, a2 in itertools.product(a1_list, a2_list):
    if a1 == a2 != 1:
        continue
    for b1, b2 in itertools.product(b1_list, b2_list):
        if b1 == b2 != 1:
            continue
        for g1, g2, g3, g4 in itertools.product(g1_list, g2_list, g3_list, g4_list):
            if g3 == g4 != 1:
                continue
            flist += [template %(dname, a1, a2, b1, b2, g1, g2, g3, g4)]

In [67]:
exist_list = [os.path.exists(f) for f in flist]
print('{} / {}'.format(sum(exist_list), len(exist_list)))

535 / 1176


In [68]:
df = pd.read_csv(flist[0], header=None)
ulist = df[1].tolist()
ytrue_old = df[0].tolist()
ytrue = [0 if ytrue_old[i] == 1 else 2 if ulist[i][0] == 's' else 1 for i in range(len(ytrue_old))]
u_sum = {u: [] for u in ulist}

miss_cnt = 0
for f in flist:
    try:
        try_df = pd.read_csv(f, header=None)
        s = dict(zip(try_df[1].tolist(), try_df[2].tolist()))
        for u in u_sum:
            u_sum[u] += [s[u]]
    except:
        miss_cnt += 1

print(miss_cnt)

641


In [69]:
y = np.array(ytrue)
X = np.array([u_sum[u] for u in ulist])

In [70]:
outfile = '../res/non-socks/rtv-%s.csv' %dname

In [71]:
models = {
    'rf': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
    'xgb': xgboost.XGBClassifier(n_estimators=100),
    'nb': sklearn.naive_bayes.BernoulliNB(),
    'log': sklearn.linear_model.LogisticRegression(solver='lbfgs'),
}

In [72]:
score = sklearn.model_selection.cross_val_predict(estimator=models['log'], X=X, y=y, cv=10, method='predict_proba')

In [73]:
df = pd.DataFrame([ytrue_old, ulist, score[:, 0]]).T
display(df.head())
df.to_csv(outfile, header=False, index=False)

,0,1,2
0,-1,u905,0.996581
1,1,u1565,0.987584
2,1,u1334,0.990841
3,1,u2045,0.990927
4,1,v1,0.995152
